In [ ]:
from google.colab import files
uploaded= files.upload()

Saving recosystem.xlsx to recosystem.xlsx


In [ ]:
import pandas as pd
import io 
dataframe = pd.read_excel(io.BytesIO(uploaded['recosystem.xlsx']))

In [ ]:
import numpy as np   
import matplotlib.pyplot as plt 
import plotly 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

In [ ]:
vect = TfidfVectorizer(ngram_range=(1,2))
tfidf = vect.fit_transform(dataframe["program_name"])

In [ ]:
def search_word(program_name):
  query_vect = vect.transform([program_name])
  similar = cosine_similarity(query_vect, tfidf).flatten()
  index = np.argpartition(similar, -5)[-5:]
  result = dataframe.iloc[index].iloc[::-1]
  return result


In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
input_name = widgets.Text(
    value='',
    description='Movie Title:'
)
list_movie = widgets.Output()

def type_movie_name(data):
    with list_movie:
        list_movie.clear_output()
        program_name = data["new"]
        if len(program_name) >= 3:
            display(search_word(program_name))

input_name.observe(type_movie_name, names='value')

display(input_name, list_movie)

Text(value='', description='Movie Title:')

Output()

In [ ]:
movie_name = 'Moana'
movie = dataframe[dataframe["program_name"] == movie_name]

In [ ]:
find_related_users = dataframe[(dataframe["program_name"] == movie_name) & (dataframe["rating"] > 3)]["user_id_maped"].unique()

In [ ]:
find_related_users

array([23732, 32443, 27549, 32439, 14274, 23425, 28506,  1630,  2406,
       27198, 17206, 18800, 12213, 14987,  2015, 26199, 15719, 12251,
        1321, 26769, 20249,  7969, 15637,  8030, 11660, 16327, 20716,
       16558, 32948, 25976, 30851, 18777, 24306, 16680, 17164, 18264,
       18617, 18127,  2120, 33871,  2219, 17700, 33705, 33745, 12041,
       26136,  9664, 24836,  4667, 18083, 15067,  5275, 20737, 15799,
       29829, 26929, 28418,  6009, 29360, 12459, 19326,  1670, 33171,
       10449, 29421,  9317,  4425, 19044, 22325, 13862, 23261, 27892,
       20226,  8228,  4074,  7294,  5094,  8682, 26704,  7519,  6274,
       20807, 33579,  3988, 33135, 12413, 29655,  7134, 15839, 18813,
         671, 28193, 27299,  9761, 25917,  6672, 27032, 14304, 24428,
       27491, 10926, 11132,  5540, 30500, 12171, 11477, 20548, 16901,
       22920,  4049,  6201, 27331,  9765,  3256,  1847, 32490,   343,
        9498, 23299,  5124, 14241, 17610, 24618,  6497, 23118,  9780,
       15965, 20014,

In [ ]:
related_user_recommendations = dataframe[(dataframe["user_id_maped"].isin(find_related_users)) & (dataframe["rating"] > 3)]["program_name"]

In [ ]:
related_user_recommendations.head(5)

68                      Zoo     Ten Years Gone
332                                     Allied
545                                  Intruders
903     Alvin and the Chipmunks: The queakquel
1064                                    Inside
Name: program_name, dtype: object

In [ ]:
related_user_recommendations = related_user_recommendations.value_counts() / len(find_related_users)

related_user_recommendations = related_user_recommendations[related_user_recommendations > .10]

In [ ]:
related_user_recommendations.head(6)

Moana                  2.850174
The Boss Baby          1.585366
Trolls                 1.104530
Teletubbies            0.933798
In the Night Garden    0.459930
Hotel Transylvania     0.425087
Name: program_name, dtype: float64

In [ ]:
total_users = dataframe[(dataframe["program_name"].isin(related_user_recommendations.index)) & (dataframe["rating"] > 3)]

In [ ]:
total_users

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre
6,6,11374,The Mermaid Princess,4,2017-06-26,Animation
41,41,18348,Ferdinand,4,2018-03-21,Animation
52,52,730,PAW Patrol Pups And The Ghost Pirate,4,2017-12-17,Animation
72,72,18960,Dora the Explorer Dora's Fantastic Gymnastics...,4,2017-12-04,Animation
125,125,29524,Dora the Explorer Dora's Easter Adventure,4,2018-04-19,Animation
...,...,...,...,...,...,...
1048515,1048515,26231,24 :00 AM - :00 AM,4,2017-12-05,Drama
1048516,1048516,23663,Dora the Explorer Dora's Moonlight Adventures,4,2017-11-18,Animation
1048533,1048533,29320,Shimmer And hine The weetest Thing,4,2017-12-04,Animation
1048549,1048549,10318,Bean,4,2017-11-19,Comedy


In [ ]:
total_user_recommendations = total_users["program_name"].value_counts() / len(total_users["user_id_maped"].unique())

In [ ]:
total_user_recommendations

Teletubbies                           0.622432
24     :00 AM - :00 AM                0.610307
In the Night Garden                   0.606938
24     :00 PM - :00 PM                0.595150
Caramel  (T)    Chapter               0.573931
                                        ...   
Farq Tawqeet    Ep.                   0.019198
Cloudy With A Chance Of Meatballs     0.018525
Lego cooby-Doo! Blowout Beach Bash    0.017514
Khalafallah    Ep.                    0.013809
Flushed Away                          0.013473
Name: program_name, Length: 132, dtype: float64

In [ ]:
recommendations_percent = pd.concat([related_user_recommendations, total_user_recommendations], axis=1)
recommendations_percent.columns = ["related_users", "total_users"]

In [ ]:
recommendations_percent

,related_users,total_users
Moana,2.850174,0.275514
The Boss Baby,1.585366,0.497474
Trolls,1.104530,0.236106
Teletubbies,0.933798,0.622432
In the Night Garden,0.459930,0.606938
...,...,...
Dora the Explorer Dora's Rescue In Mermaid Kingdom,0.101045,0.130010
Dora And Friends: Into The City We ave The Music,0.101045,0.073425
Dora the Explorer Vamos A Pintar!,0.101045,0.116874
Dora the Explorer Dora And Diego's Amazing Animal Circus ...,0.101045,0.113506


In [ ]:
recommendations_percent["score"] = recommendations_percent["related_users"] / recommendations_percent["total_users"]


In [ ]:
recommendations_percent = recommendations_percent.sort_values("score", ascending=False)


In [ ]:
recommendations_percent

,related_users,total_users,score
Moana,2.850174,0.275514,10.344948
Khalafallah Ep.,0.128920,0.013809,9.335685
Flushed Away,0.101045,0.013473,7.500087
Toy tory,0.153310,0.022567,6.793697
The Lego Batman Movie,0.184669,0.027956,6.605810
...,...,...,...
The Expendables,0.132404,0.258673,0.511859
Caramel (T) Chapter,0.212544,0.573931,0.370330
Blaze And The Monster Machines Blaze Of Glory (Part ),0.177700,0.480970,0.369462
24 :00 PM - :00 PM,0.135889,0.595150,0.228327


In [ ]:
recommendations_percent.merge(dataframe, left_index=True, right_on="program_name")

,related_users,total_users,score,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre
1,2.850174,0.275514,10.344948,1,7946,Moana,1,2017-05-21,Animation
18,2.850174,0.275514,10.344948,18,26546,Moana,1,2017-06-25,Animation
19,2.850174,0.275514,10.344948,19,20306,Moana,1,2017-06-23,Animation
48,2.850174,0.275514,10.344948,48,28687,Moana,2,2017-05-30,Animation
191,2.850174,0.275514,10.344948,191,16739,Moana,1,2017-05-27,Animation
...,...,...,...,...,...,...,...,...,...
1047193,0.128920,0.610307,0.211238,1047193,161,24 :00 AM - :00 AM,4,2017-12-06,Drama
1047231,0.128920,0.610307,0.211238,1047231,194,24 :00 AM - :00 AM,4,2017-10-21,Drama
1047729,0.128920,0.610307,0.211238,1047729,29277,24 :00 AM - :00 AM,1,2017-11-11,Drama
1047732,0.128920,0.610307,0.211238,1047732,15480,24 :00 AM - :00 AM,4,2017-09-16,Drama
